# Your First Image Classifier: Using k-NN, CNN and MLP to Classify Images
# Exploratory Data Analysis and Pre-processing

The purpose of this dataset is to correctly classify an image as containing a dog, cat, or panda.
Containing only 3,000 images, the Animals dataset is meant to be another **introductory** dataset
that we can quickly train a KNN, MLP or CNN model and obtain initial results that has potential to be used as a baseline. 

Let's take the following steps:

1. Exploratory Data Analysis (EDA)
2. Pre-processing

<center><img width="600" src="https://drive.google.com/uc?export=view&id=1a-nyAPNPiVh-Xb2Pu2t2p-BhSvHJS0pO"></center>

## Step 01: Setup

Start out by installing the experiment tracking library and setting up your free W&B account:


*   **pip install wandb** – Install the W&B library
*   **import wandb** – Import the wandb library
*   **wandb login** – Login to your W&B account so you can log all your metrics in one place

In [ ]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 162 kB 33.9 MB/s 
     |████████████████████████████████| 182 kB 44.3 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 162 kB 42.8 MB/s 
     |████████████████████████████████| 158 kB 42.3 MB/s 
     |████████████████████████████████| 157 kB 33.5 MB/s 
     |████████████████████████████████| 157 kB 46.6 MB/s 
     |████████████████████████████████| 157 kB 41.0 MB/s 
     |████████████████████████████████| 157 kB 32.7 MB/s 
     |████████████████████████████████| 157 kB 38.6 MB/s 
     |████████████████████████████████| 157 kB 39.2 MB/s 
     |████████████████████████████████| 157 kB 47.1 MB/s 
     |████████████████████████████████| 156 kB 54.5 MB/s 
ERROR: Operation cancelled by user


In [ ]:
import wandb
wandb.login()

### Import Packages

In [ ]:
# import the necessary packages
from imutils import paths
import logging
import os
import cv2
import numpy as np
import joblib

In [ ]:
# configure logging
# reference for a logging obj
logger = logging.getLogger()

# set level of logging
logger.setLevel(logging.INFO)

# create handlers
c_handler = logging.StreamHandler()
c_format = logging.Formatter(fmt="%(asctime)s %(message)s",datefmt='%d-%m-%Y %H:%M:%S')
c_handler.setFormatter(c_format)

# add handler to the logger
logger.handlers[0] = c_handler

## Step 02 EDA

In [ ]:
# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
	"dataset": "animals",
  "project_name": "first_image_classifier",
  "artifact_name": "animals_raw_data:latest",
  "eda_name": "eda_animals"
}

In [ ]:
# open the W&B project created in the Fetch step
run = wandb.init(entity="thaisaraujom",project=args["project_name"], job_type="preprocessing")

# download the raw data from W&B
raw_data = run.use_artifact(args["artifact_name"])
data_dir = raw_data.download()
logger.info("Path: {}".format(data_dir))

wandb: Currently logged in as: thaisaraujom. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact animals_raw_data:latest, 187.97MB. 3000 files... 
wandb:   3000 of 3000 files downloaded.  
Done. 0:0:31.9
16-10-2022 04:25:15 Path: ./artifacts/animals_raw_data:v0


In [ ]:
# create a table with columns we want to track/compare
preview_dt = wandb.Table(columns=["id", "image", "label","size"])

# create a new artifact to store the EDA data
eda_data = wandb.Artifact(args["eda_name"], type="eda_data")

In [ ]:
# grab the list of images that we'll be describing
imagePaths = list(paths.list_images(data_dir))

# append all images to the artifact
for img in imagePaths:
  "img example: ./artifacts/animals_raw_data:v0/dogs/dogs_00892.jpg"
  label = img.split(os.path.sep)
  image = cv2.imread(img)
  preview_dt.add_data(label[-1], wandb.Image(img), label[-2], str(image.shape[0]) + " X " + str(image.shape[1]))

In [ ]:
# save artifact to W&B
eda_data.add(preview_dt, "EDA_Table")
run.log_artifact(eda_data)

## Step 03 - Clean Data

In [ ]:
# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
	"dataset": "clean_data",
  "label": "label",
  "project_name": "first_image_classifier",
  "artifact_name": "animals_raw_data:latest"
}

In [ ]:
# download the raw data from W&B
raw_data = run.use_artifact(args["artifact_name"])
data_dir = raw_data.download()
logger.info("Path: {}".format(data_dir))

wandb: Downloading large artifact animals_raw_data:latest, 187.97MB. 3000 files... 
wandb:   3000 of 3000 files downloaded.  
Done. 0:0:0.5
16-10-2022 04:25:52 Path: ./artifacts/animals_raw_data:v0


In [ ]:
# a basic simple preprocessor
class SimplePreprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# store the target image width, height, and interpolation
		# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter

	def preprocess(self, image):
		# resize the image to a fixed size, ignoring the aspect
		# ratio
		return cv2.resize(image, (self.width, self.height),interpolation=self.inter)

In [ ]:
# Building an image loader
class SimpleDatasetLoader:
  def __init__(self, preprocessors=None, logger=None):
		# store the image preprocessor
    self.preprocessors = preprocessors
    self.logger = logger

		# if the preprocessors are None, initialize them as an
		# empty list
    if self.preprocessors is None:
      self.preprocessors = []

  def load(self, imagePaths, verbose=-1):
		# initialize the list of features and labels
    data = []
    labels = []

		# loop over the input images
    for (i, imagePath) in enumerate(imagePaths):
			# load the image and extract the class label assuming
			# that our path has the following format:
			# /path/to/dataset/{class}/{image}.jpg
			# e.g "img example: ./artifacts/animals_raw_data:v0/dogs/dogs_00892.jpg"
			# imagePath.split(os.path.sep)[-2] will return "dogs"
      image = cv2.imread(imagePath)
      label = imagePath.split(os.path.sep)[-2]

      # check to see if our preprocessors are not None
      if self.preprocessors is not None:
				# loop over the preprocessors and apply each to
				# the image
        for p in self.preprocessors:
          image = p.preprocess(image)

			# treat our processed image as a "feature vector"
			# by updating the data list followed by the labels
      data.append(image)
      labels.append(label)
   
			# show an update every `verbose` images
      if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
        logger.info("[INFO] processed {}/{}".format(i + 1,len(imagePaths)))

		# return a tuple of the data and labels
    return (np.array(data), np.array(labels))

In [ ]:
# grab the list of images that we'll be describing
logger.info("[INFO] preprocessing images...")
imagePaths = list(paths.list_images(data_dir))

# initialize the image preprocessor, load the dataset from disk,
# and reshape the data matrix
sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp],logger=logger)
(data, labels) = sdl.load(imagePaths, verbose=500)
# 32 x 32 x 3 = 3072
data = data.reshape((data.shape[0], 3072))

# show some information on memory consumption of the images
logger.info("[INFO] features matrix: {:.1f}MB".format(data.nbytes / (1024 * 1024)))

16-10-2022 04:25:55 [INFO] preprocessing images...
16-10-2022 04:25:59 [INFO] processed 500/3000
16-10-2022 04:26:02 [INFO] processed 1000/3000
16-10-2022 04:26:05 [INFO] processed 1500/3000
16-10-2022 04:26:08 [INFO] processed 2000/3000
16-10-2022 04:26:17 [INFO] processed 2500/3000
16-10-2022 04:26:24 [INFO] processed 3000/3000
16-10-2022 04:26:24 [INFO] features matrix: 8.8MB


In [ ]:
logger.info("Data shape: {}".format(data.shape))
logger.info("Label shape: {}".format(labels.shape))

16-10-2022 04:26:24 Data shape: (3000, 3072)
16-10-2022 04:26:24 Label shape: (3000,)


In [ ]:
logger.info("Dumping the clean data artifacts to disk")
# Save the feature artifacts using joblib
joblib.dump(data, args["dataset"])

# Save the target using joblib
joblib.dump(labels, args["label"])

16-10-2022 04:26:24 Dumping the clean data artifacts to disk


['label']

In [ ]:
# clean data artifact
artifact = wandb.Artifact(args["dataset"],
                          type="CLEAN_DATA",
                          description="A json file representing the clean and preprocessed data"
                          )

logger.info("Logging clean data artifact")
artifact.add_file(args["dataset"])
run.log_artifact(artifact)

16-10-2022 04:26:24 Logging clean data artifact


In [ ]:
# clean label artifact
artifact = wandb.Artifact(args["label"],
                          type="CLEAN_DATA",
                          description="A json file representing the clean label"
                          )

logger.info("Logging clean label artifact")
artifact.add_file(args["label"])
run.log_artifact(artifact)

16-10-2022 04:26:25 Logging clean label artifact


In [ ]:
run.finish()